In [2]:
import pandas as pd
from lib.dicts.index import Index
from lib.data.data import DataImporter, DataExporter
from lib.data.data_parser import ProcessData
from lib.backtest.backtest import BacktestManager
from lib.forecast.forecast import *

In [3]:
indexes = ['SAN.MC', 'BBVA.MC']#, 'SAB.MC', '^IBEX', 'BBVAE.MC', 'XTC5.MI', 'EURUSD=X']
input_method = 'csv'
start_get_data = '2021-07-30'
end_get_data = '2024-07-30'
start_calculation_date = '2024-06-30'
end_calculation_date = '2024-07-30'
confidence_level = 0.975
horizons = [1, 10]

In [11]:
forecast = pd.read_csv(f'input/forecast.csv', sep=';')
index_instance = Index(indexes)
data_instance = DataImporter(indexes, start_get_data, end_get_data, input_method)
process_data_instance = ProcessData(index_instance, data_instance)
process_data_instance.fill_index_dict(
    confidence_level,
    start_calculation_date,
    end_calculation_date
)

In [12]:
forecast['VOLATILITY'] = forecast['VOLATILITY'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

In [13]:
forecast

,Date,VOLATILITY,Index,Volatility,Horizon,Model
0,2024-07-25 00:00:00,0.013911,SAN.MC,STD,10,PERCEPTRON
1,2024-07-26 00:00:00,0.013782,SAN.MC,STD,10,PERCEPTRON
2,2024-07-29 00:00:00,0.013864,SAN.MC,STD,10,PERCEPTRON
3,2024-07-25 00:00:00,0.015073,SAN.MC,STD,10,RANDOM_FOREST
4,2024-07-26 00:00:00,0.014744,SAN.MC,STD,10,RANDOM_FOREST
...,...,...,...,...,...,...
103,2024-07-26 00:00:00,0.018210,BBVA.MC,GJR_GARCH,10,LSTM
104,2024-07-29 00:00:00,0.018129,BBVA.MC,GJR_GARCH,10,LSTM
105,2024-07-25 00:00:00,0.015181,BBVA.MC,GJR_GARCH,10,RANDOM_FOREST
106,2024-07-26 00:00:00,0.017607,BBVA.MC,GJR_GARCH,10,RANDOM_FOREST


In [21]:
final_dict = {}

In [23]:
for (index, volatility, horizon, model), group in forecast.groupby(['Index', 'Volatility', 'Horizon', 'Model']):
    # Asegurar que las claves existen en el diccionario final
    final_dict.setdefault(index, {}).setdefault(volatility, {}).setdefault(horizon, {})
    
    # Crear un DataFrame con 'Date' como índice y 'VOLATILITY' como columna
    temp_df = group.set_index('Date')[['VOLATILITY']]
    
    # Asignar el DataFrame al diccionario final en la estructura requerida
    final_dict[index][volatility][horizon][model] = temp_df

In [24]:
final_dict

{'BBVA.MC': {'EWMA': {1: {'LSTM':                      VOLATILITY
    Date                           
    2024-07-25 00:00:00    0.013749
    2024-07-26 00:00:00    0.013578
    2024-07-29 00:00:00    0.013308,
    'PERCEPTRON':                      VOLATILITY
    Date                           
    2024-07-25 00:00:00    0.013721
    2024-07-26 00:00:00    0.013277
    2024-07-29 00:00:00    0.013121,
    'RANDOM_FOREST':                      VOLATILITY
    Date                           
    2024-07-25 00:00:00    0.013577
    2024-07-26 00:00:00    0.013233
    2024-07-29 00:00:00    0.013110},
   10: {'LSTM':                      VOLATILITY
    Date                           
    2024-07-25 00:00:00    0.016435
    2024-07-26 00:00:00    0.016202
    2024-07-29 00:00:00    0.015931,
    'PERCEPTRON':                      VOLATILITY
    Date                           
    2024-07-25 00:00:00    0.016217
    2024-07-26 00:00:00    0.015904
    2024-07-29 00:00:00    0.016194,
    'RA

In [5]:
data_dict = index_instance.get_index_dict()
forecast_dict = index_instance.get_forecast_dict()

In [6]:
forecast_instance = Forecast(data_dict, forecast_dict, start_calculation_date, end_calculation_date, horizons)
forecast_instance.run_forecast()

Progreso global forecasting: 100.00%. Ejecutado con 3/3 volatilidades de 2/2 índices 

In [7]:
forecast_dict

{'SAN.MC': {'STD': {1: {'PERCEPTRON':             VOLATILITY
    Date                  
    2024-07-01    0.014000
    2024-07-02    0.013652
    2024-07-03    0.013643
    2024-07-04    0.013694
    2024-07-05    0.013665
    2024-07-08    0.013854
    2024-07-09    0.014067
    2024-07-10    0.014209
    2024-07-11    0.014369
    2024-07-12    0.014449
    2024-07-15    0.014505
    2024-07-16    0.014407
    2024-07-17    0.014629
    2024-07-18    0.014581
    2024-07-19    0.014722
    2024-07-22    0.014608
    2024-07-23    0.014623
    2024-07-24    0.014639
    2024-07-25    0.014734
    2024-07-26    0.014901
    2024-07-29    0.015169
    2024-07-30    0.015264,
    'LSTM':             VOLATILITY
    Date                  
    2024-07-01    0.012597
    2024-07-02    0.012597
    2024-07-03    0.012597
    2024-07-04    0.012597
    2024-07-05    0.012597
    2024-07-08    0.012597
    2024-07-09    0.012597
    2024-07-10    0.012597
    2024-07-11    0.012597
    2024-07-